# HTML-to-JSON 구조화 데이터 변환 튜토리얼

이 튜토리얼에서는 **vLLM**과 **LLaMA-Factory**를 활용하여 HTML 콘텐츠를 구조화된 JSON 형태로 변환하는 방법을 학습합니다.

## 1. vLLM 서버 실행 및 연결 테스트

먼저 vLLM이 정상적으로 작동하는지 확인해보겠습니다. 우리는 [Llama-3.1-8B-Instruct](https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct) 모델을 사용합니다.

### 사전 준비사항
1. 🤗 Hugging Face에 가입하여 액세스 토큰을 발급받으세요
2. [Llama-3.1-8B-Instruct 페이지](https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct)에서 모델 사용 등록을 완료하세요

### vLLM 서버 실행 명령어 설명
- `HF_TOKEN`: 발급받은 Hugging Face 토큰을 입력하세요
- `CUDA_VISIBLE_DEVICES`: 사용할 GPU 번호 (전체 GPU 사용시 생략 가능)  
- `-tp`: 텐서 병렬화에 사용할 GPU 개수를 지정합니다

**⚠️ 중요**: 아래 명령어를 별도의 터미널에서 실행하세요.

In [ ]:
export HF_TOKEN=<허깅페이스 토큰>
CUDA_VISIBLE_DEVICES=0,1 vllm serve meta-llama/Llama-3.1-8B-Instruct -tp 2

### ✅ vLLM 서버 실행 확인

터미널에서 다음과 같은 로그가 출력되면 vLLM 서버가 정상적으로 시작된 것입니다:

```log
INFO:     Started server process [2609659]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
```

이제 **Langchain**을 사용하여 실행 중인 vLLM 서버에 간단한 요청을 보내 정상 작동을 확인해보겠습니다.

In [ ]:
# langchain-openai 라이브러리 임포트
from langchain_openai import ChatOpenAI

In [ ]:
# vLLM 서버에 연결 (OpenAI 호환 API 사용)
llm = ChatOpenAI(
    base_url="http://localhost:8000/v1",  # vLLM 서버 주소
    api_key="EMPTY",  # vLLM에서는 임의의 값 사용
    model="meta-llama/Llama-3.1-8B-Instruct",  # 모델 이름
    temperature=0.1
)

# 채팅 메시지 보내기
from langchain_core.messages import HumanMessage

messages = [HumanMessage(content="안녕 너는 누구야?")]
response = llm.invoke(messages)

print("AI 응답:", response.content)

### 📊 서버 요청 처리 확인

위 코드를 실행한 후, vLLM을 실행시킨 터미널에서 다음과 같은 로그가 출력되면 요청이 성공적으로 처리된 것입니다:

```log
INFO 08-13 23:26:51 [logger.py:41] Received request chatcmpl-543715af49a8443da65b81033b0cd2de: 
prompt: '<|begin_of_text|><|start_header_id|>system<|end_header_id|>...
INFO 08-13 23:26:51 [async_llm.py:269] Added request chatcmpl-543715af49a8443da65b81033b0cd2de.
INFO:     127.0.0.1:39490 - "POST /v1/chat/completions HTTP/1.1" 200 OK
INFO 08-13 23:26:56 [loggers.py:122] Engine 000: Avg prompt throughput: 4.2 tokens/s, 
Avg generation throughput: 4.7 tokens/s, Running: 0 reqs, Waiting: 0 reqs, 
GPU KV cache usage: 0.0%, Prefix cache hit rate: 0.0%
```

이 로그를 통해 요청 처리 속도, GPU 메모리 사용량 등을 모니터링할 수 있습니다.

## 2. 실습 데이터셋 및 목표 설정

### 📋 사용 데이터셋
이번 실습에서는 [mdhasnainali/job-html-to-json](https://huggingface.co/datasets/mdhasnainali/job-html-to-json) 데이터셋을 활용합니다.

### 🎯 실습 목표
HTML 형태의 채용공고 데이터를 구조화된 JSON 형태로 변환하는 것이 목표입니다.

**변환 예시:**
- **입력**: `data/formatting/input.html` (HTML 채용공고)
- **출력**: `data/formatting/target.json` (구조화된 JSON)

먼저 입력 데이터와 목표 출력 형태를 살펴보겠습니다.

In [ ]:
# 0.html 파일 내용 출력
html_file_path = "data/formatting/input.html"
with open(html_file_path, 'r', encoding='utf-8') as f:
    html_content = f.read()
print(html_content)

In [ ]:
# 0.json 파일 내용 출력
import json

json_file_path = "data/formatting/target.json"
with open(json_file_path, 'r', encoding='utf-8') as f:
    json_content = f.read()
print(json_content)

### 📝 목표 JSON 구조 정의

우리가 생성하고자 하는 JSON의 구조를 **Pydantic**을 사용하여 정의합니다. 이 스키마는 채용공고의 모든 핵심 정보를 체계적으로 구조화합니다.

다음 코드는 채용공고 JSON의 상세한 스키마를 정의합니다:

In [ ]:
from pydantic import BaseModel
import json

class ApplicationInfo(BaseModel):
    apply_url: str
    contact_email: str
    deadline: str


class Location(BaseModel):
    city: str
    state: str | None
    country: str | None
    remote: bool | None
    hybrid: bool | None


class Qualifications(BaseModel):
    education_level: str | None
    fields_of_study: str | None
    certifications: list[str] | None


class Salary(BaseModel):
    currency: str | None
    min: float | None
    max: float | None
    period: str | None


class YearsOfExperience(BaseModel):
    min: float | None
    max: float | None


class JobPosting(BaseModel):
    job_id: str
    title: str
    department: str
    employment_type: str
    experience_level: str
    posted_date: str
    work_schedule: str

    location: Location
    application_info: ApplicationInfo
    salary: Salary
    years_of_experience: YearsOfExperience

    requirements: list[str]
    responsibilities: list[str]
    nice_to_have: list[str]
    qualifications: Qualifications
    recruitment_process: list[str]
    programming_languages: list[str]
    tools: list[str]
    databases: list[str]
    cloud_providers: list[str]
    language_requirements: list[str]
    benefits: list[str]

print(json.dumps(JobPosting.model_json_schema(), indent=2))

## 3. Zero-shot 정확도 평가

### 🧪 실험 1: 기본 Zero-shot 예측

먼저 **fine-tuning 없이** 기본 LLaMA 모델이 HTML을 JSON으로 얼마나 잘 변환할 수 있는지 확인해보겠습니다.

In [ ]:
# Zero-shot HTML to JSON 변환
from langchain_core.messages import SystemMessage, HumanMessage
import json

# JSON schema 가져오기
json_schema = json.dumps(JobPosting.model_json_schema(), indent=2)

# System prompt 생성
system_prompt = f"""주어진 HTML 입력을 JSON으로 바꾸시오. 이때 JSON schema는 다음과 같다:
{json_schema}

HTML에서 정보를 추출하여 위 스키마에 맞는 JSON 형태로 변환하시오. 다른 텍스트는 포함하지 말고 json을 바로 출력하시오."""

# 메시지 구성
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=html_content)
]

# LLM에 요청
response = llm.invoke(messages)

print(response.content)

### 🔧 실험 2: Constrained Decoding으로 Zero-shot 예측

기본 zero-shot 출력에서는 다음과 같은 문제가 발생할 수 있습니다:
- 불필요한 텍스트가 포함됨
- JSON 구조가 올바르지 않음
- 스키마를 준수하지 않음

이러한 문제를 해결하기 위해 **Constrained Decoding**을 사용하여 출력을 JSON 스키마에 강제로 맞춰보겠습니다.

In [ ]:
# Zero-shot HTML to JSON 변환
from langchain_core.messages import SystemMessage, HumanMessage
import json

# JSON schema 가져오기
json_schema = json.dumps(JobPosting.model_json_schema(), indent=2)

# System prompt 생성
system_prompt = f"""주어진 HTML 입력을 JSON으로 바꾸시오. 이때 JSON schema는 다음과 같다:
{json_schema}

HTML에서 정보를 추출하여 위 스키마에 맞는 JSON 형태로 변환하시오. 다른 텍스트는 포함하지 말고 json을 바로 출력하시오."""

# 메시지 구성
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=html_content)
]

# Constrained Decoding이 적용
constrained_llm = ChatOpenAI(
    base_url="http://localhost:8000/v1",  # vLLM 서버 주소
    api_key="EMPTY",  # vLLM에서는 임의의 값 사용
    model="meta-llama/Llama-3.1-8B-Instruct",  # 모델 이름
    temperature=0.1,
    extra_body={
        "guided_json": JobPosting.model_json_schema() 
    }
)

# LLM에 요청
response = constrained_llm.invoke(messages)

obj = json.loads(response.content)

import pprint
pprint.pprint(obj)

### 📈 Zero-shot 결과 분석

Constrained decoding을 통해 JSON 형태의 출력을 얻을 수 있었지만, 실제 정답(ground truth)과 세부적으로 비교해보면 정확도에 한계가 있습니다. 

이제 **Fine-tuning**을 통해 성능을 개선해보겠습니다.

## 4. LLaMA-Factory를 활용한 Fine-tuning

### 🛠️ 학습 데이터 준비

이제 **LLaMA-Factory**를 사용하여 HTML-to-JSON 변환 성능을 향상시키기 위한 fine-tuning을 진행합니다.

먼저 다음 명령어를 실행하여 학습용 데이터셋을 생성하겠습니다.

**참고 자료**: [LLaMA-Factory 데이터셋 설정 가이드](https://github.com/hiyouga/LLaMA-Factory/tree/main/data)

In [ ]:
# 데이터 생성 코드
python -m demo.create_formatting_dataset
# 생성된 데이터 확인
head -16 data/formatting/train.json

### 📊 데이터셋 토큰 길이 분석

Fine-tuning을 효과적으로 수행하기 위해 생성된 데이터셋의 토큰 길이 분포를 분석합니다. 이를 통해 적절한 `cutoff_len` 설정을 결정할 수 있습니다.

In [ ]:
# tokenizer 로드 및 토큰 길이 측정 (배치 처리)
from transformers import AutoTokenizer
import json
import numpy as np

# Llama-3.1-8B-Instruct tokenizer 로드
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")

# 데이터 로드
with open("data/formatting/train.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

print(f"총 데이터 개수: {len(data)}")

# 모든 메시지를 chat template으로 변환
print("Chat template 적용 중...")
formatted_texts = []

for i, item in enumerate(data):
    messages = item["messages"]
    
    # apply_chat_template 사용
    formatted_text = tokenizer.apply_chat_template(
        messages, 
        tokenize=False,
        add_generation_prompt=False
    )
    formatted_texts.append(formatted_text)
    
    # 진행상황 출력
    if (i + 1) % 1000 == 0:
        print(f"진행: {i + 1}/{len(data)}")

print("배치 토큰화 시작...")

# 배치 토큰화 (메모리를 고려하여 배치 크기 조절)
batch_size = 100
token_lengths = []

for i in range(0, len(formatted_texts), batch_size):
    batch_texts = formatted_texts[i:i + batch_size]
    
    # 배치 토큰화
    batch_tokens = tokenizer(
        batch_texts,
        add_special_tokens=False,
        return_attention_mask=False,
        return_token_type_ids=False,
        padding=False,
        truncation=False
    )
    
    # 각 텍스트의 토큰 길이 계산
    for tokens in batch_tokens['input_ids']:
        token_lengths.append(len(tokens))
    
    # 진행상황 출력
    if (i + batch_size) % 1000 == 0:
        print(f"토큰화 진행: {min(i + batch_size, len(formatted_texts))}/{len(formatted_texts)}")

print(f"토큰화 완료! 총 {len(token_lengths)}개 샘플 처리됨")

# 통계 계산
token_lengths = np.array(token_lengths)
print(f"\n=== 토큰 길이 통계 ===")
print(f"평균 토큰 길이: {np.mean(token_lengths):.2f}")
print(f"중간값 토큰 길이: {np.median(token_lengths):.2f}")
print(f"최소 토큰 길이: {np.min(token_lengths)}")
print(f"최대 토큰 길이: {np.max(token_lengths)}")
print(f"표준편차: {np.std(token_lengths):.2f}")

# 분위수 정보
print(f"\n=== 토큰 길이 분위수 ===")
print(f"25% 분위수: {np.percentile(token_lengths, 25):.2f}")
print(f"50% 분위수 (중간값): {np.percentile(token_lengths, 50):.2f}")
print(f"75% 분위수: {np.percentile(token_lengths, 75):.2f}")
print(f"90% 분위수: {np.percentile(token_lengths, 90):.2f}")
print(f"95% 분위수: {np.percentile(token_lengths, 95):.2f}")
print(f"99% 분위수: {np.percentile(token_lengths, 99):.2f}")

# 토큰 길이별 분포
print(f"\n=== 토큰 길이 분포 ===")
bins = [0, 1000, 2000, 4000, 8000, 16000, 32000, float('inf')]
labels = ['0-1K', '1K-2K', '2K-4K', '4K-8K', '8K-16K', '16K-32K', '32K+']

for i in range(len(bins)-1):
    count = np.sum((token_lengths >= bins[i]) & (token_lengths < bins[i+1]))
    percentage = count / len(token_lengths) * 100
    print(f"{labels[i]}: {count}개 ({percentage:.1f}%)")

### ⚙️ 학습 설정 결정

분석 결과, 32K 토큰을 초과하는 샘플이 없음을 확인했습니다. 따라서 `cutoff_len=32768`로 설정하여 모든 데이터를 효율적으로 활용할 수 있습니다.

이제 fine-tuning을 시작합니다:

In [ ]:
# demo/formatting.yaml 파일을 확인해보자.
CUDA_VISIBLE_DEVICES=0,1 llamafactory-cli train demo/formatting.yaml

### 🚀 Fine-tuned 모델(LoRA) 서빙

학습이 완료되면 vLLM에 **LoRA 어댑터**를 추가하여 fine-tuned 모델을 서빙합니다. 

다음 명령어로 LoRA가 적용된 vLLM 서버를 실행하세요:

In [ ]:
export HF_TOKEN=<허깅페이스 토큰>
CUDA_VISIBLE_DEVICES=0,1 vllm serve meta-llama/Llama-3.1-8B-Instruct -tp 2 --enable-lora --lora-modules formatting=$PWD/save/llama-lora

## 5. 성능 비교 및 평가

### 🔍 Base Model vs Fine-tuned Model 비교

vLLM 서버가 LoRA와 함께 실행되면, 다음 두 모델의 성능을 직접 비교해보겠습니다:

1. **Base Model**: 원본 Llama-3.1-8B-Instruct
2. **Fine-tuned Model**: LoRA가 적용된 모델

동일한 입력에 대한 두 모델의 출력을 비교하여 fine-tuning의 효과를 확인합니다.

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage
import json

# JSON schema 가져오기
json_schema = json.dumps(JobPosting.model_json_schema(), indent=2)

# System prompt 생성
system_prompt = f"""주어진 HTML 입력을 JSON으로 바꾸시오. 이때 JSON schema는 다음과 같다:
{json_schema}

HTML에서 정보를 추출하여 위 스키마에 맞는 JSON 형태로 변환하시오. 다른 텍스트는 포함하지 말고 json을 바로 출력하시오."""

# 메시지 구성
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=html_content)
]

# 기존 LLM
constrained_llm = ChatOpenAI(
    base_url="http://localhost:8000/v1",  # vLLM 서버 주소
    api_key="EMPTY",  # vLLM에서는 임의의 값 사용
    model="meta-llama/Llama-3.1-8B-Instruct",  # 모델 이름
    temperature=0.1,
    extra_body={
        "guided_json": JobPosting.model_json_schema() 
    }
)

# 메시지 구성
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content="".join(html_content.split()))          # LoRA에 사용된 학습데이터는, 학습 데이터에 줄 나눔이 없다.
]

# LoRA가 적용된 LLM
constrained_lora_llm = ChatOpenAI(
    base_url="http://localhost:8000/v1",  # vLLM 서버 주소
    api_key="EMPTY",  # vLLM에서는 임의의 값 사용
    model="formatting",  # LoRA 버전
    temperature=0.1,
    extra_body={
        "guided_json": JobPosting.model_json_schema() 
    }
)

import pprint
# 그냥 LLM에 요청
response = constrained_llm.invoke(messages)

naive_output = json.loads(response.content)

print("기존 LLM 응답:")
pprint.pprint(naive_output)


# Lora LLM에 요청
response = constrained_lora_llm.invoke(messages)

lora_output = json.loads(response.content)

print("LoRA LLM 응답:")
pprint.pprint(lora_output)

### 📏 정량적 성능 평가

두 모델의 정확도를 간단하게 비교하기 위해 **텍스트 유사성**을 측정해봅시다.

**평가 방법:**
- 정답 JSON과 각 모델의 출력 JSON을 문자열로 변환
- `difflib.SequenceMatcher`를 사용하여 유사성 점수 계산
- 0.0 (완전히 다름) ~ 1.0 (완전히 같음) 범위로 평가

이를 통해 fine-tuning이 실제로 성능 향상에 기여했는지 확인할 수 있습니다.

In [ ]:
# 간단한 유사성 비교
import json
from difflib import SequenceMatcher

# JSON을 문자열로 변환하여 유사성 측정
target_str = json.dumps(json.loads(json_content), sort_keys=True, ensure_ascii=False)
lora_str = json.dumps(lora_output, sort_keys=True, ensure_ascii=False)
naive_str = json.dumps(naive_output, sort_keys=True, ensure_ascii=False)

# 유사성 계산
lora_similarity = SequenceMatcher(None, target_str, lora_str).ratio()
naive_similarity = SequenceMatcher(None, target_str, naive_str).ratio()

print("=== 유사성 비교 결과 ===")
print(f"LoRA 모델 유사성:  {lora_similarity:.4f}")
print(f"Naive 모델 유사성: {naive_similarity:.4f}")
print(f"차이: {lora_similarity - naive_similarity:.4f}")

if lora_similarity > naive_similarity:
    improvement = ((lora_similarity - naive_similarity) / naive_similarity) * 100
    print(f"✅ LoRA 모델이 {improvement:.2f}% 더 우수합니다!")
else:
    decline = ((naive_similarity - lora_similarity) / naive_similarity) * 100
    print(f"❌ Naive 모델이 {decline:.2f}% 더 우수합니다!")